In [2]:
%run 'template.ipynb'

In [56]:
# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a writer of articles about home flower care."
            "Write this article in Russia language."
            "Add in header ## symbols and bold so **word** important words"
            "Don't ask about answer."
            "No more then 200 words on article.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [4]:
llm = ChatOllama(
    model="qwen2.5",
    temperature=0.05,
)

In [6]:
from sqlmodel import SQLModel, Relationship, Field
from sqlalchemy import Column, DateTime, text
from datetime import datetime
from uuid import UUID, uuid4


class Article(SQLModel, table=True):
    __tablename__ = "articles"
    id: UUID = Field(primary_key=True, default_factory=uuid4)
    text: str

    created_at: datetime = Field(
        sa_column=Column(
            DateTime(timezone=True),
            nullable=False,
            server_default=text("CURRENT_TIMESTAMP"),
        )
    )
    updated_at: datetime = Field(
        sa_column=Column(
            DateTime(timezone=True),
            nullable=False,
            server_default=text("CURRENT_TIMESTAMP"),
            onupdate=text("CURRENT_TIMESTAMP"),
        )
    )

    plant_id: UUID = Field(foreign_key="plants.id")

In [8]:
from sqlmodel import SQLModel, Field
from uuid import UUID, uuid4
from datetime import date


class Plant(SQLModel, table=True):
    __tablename__ = "plants"
    # __table_args__ = {"extend_existing": True}
    id: UUID = Field(primary_key=True, default_factory=uuid4)
    origin_name: str
    name: str | None
    bibliography: str
    slug: str
    year: date
    image_url: str
    family: str
    genus: str
    rank: str

In [45]:
from sqlmodel import select


async with session_maker() as session:
    plants = (
        await session.exec(select(Plant).where(Plant.name != None).limit(10))
    ).all()

DBAPIError: (sqlalchemy.dialects.postgresql.asyncpg.Error) <class 'asyncpg.exceptions.ConnectionDoesNotExistError'>: connection was closed in the middle of operation
[SQL: SELECT plants.id, plants.origin_name, plants.name, plants.bibliography, plants.slug, plants.year, plants.image_url, plants.family, plants.genus, plants.rank 
FROM plants 
WHERE plants.name IS NOT NULL 
 LIMIT $1::INTEGER]
[parameters: (10,)]
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [57]:
from tqdm import tqdm

df = pd.DataFrame(columns=["text"])
for plant in tqdm(plants, total=len(plants), desc="Anchoring", unit="page"):
    text = plant.origin_name
    prompt = prompt_template.invoke({"text": text})
    response = llm.invoke(prompt)
    print(response)
    df.loc[plant.id] = (response.content,)

Anchoring:  10%|█         | 1/10 [01:04<09:44, 64.91s/page]

content='## Матграсс: Уход за домашним растением\n\n**Матграсс**, или матовый газон, является популярным выбором для декорирования интерьера благодаря своей устойчивости и низким требованиям к уходу. Этот вид травы хорошо подходит для тех, кто предпочитает минималистичный стиль оформления.\n\n**Условия выращивания:**\n- **Освещение:** Матграсс предпочитает яркий свет, но может расти и в полутени.\n- **Влажность:** Поддерживайте среднюю влажность воздуха, избегая пересыхания почвы.\n\n**Уход за матграссом:**\n- Регулярно подкармливайте растение специальными удобрениями для травы.\n- Обрезайте при необходимости, чтобы поддерживать форму и здоровье растения.\n- Избегайте переливания воды, так как это может привести к загниванию корней.\n\n**Преимущества:**\nМатграсс легко ухаживается, не требует частого полива или обрезки. Он также помогает создать спокойную и расслабляющую атмосферу в доме благодаря своей неприхотливости и эстетической привлекательности.\n\n**Заключение:**\nМатграсс — от

Anchoring:  20%|██        | 2/10 [02:50<11:50, 88.84s/page]

content='## **Комнатное растение: Плантейн**\n\n**Плантейн**, или комнатный плантейн, является популярным выбором для начинающих цветоводов благодаря своей простоте в уходе и декоративности. Этот вид растения отличается крупными листьями зеленого цвета с серыми прожилками, которые придают ему уникальный внешний вид.\n\nДля успешного выращивания плантейна важно соблюдать несколько основных правил:\n\n- **Освещение**: Предпочтительное место — светлое окно без прямых солнечных лучей.\n- **Влажность**: Регулярный опрыскивание листьев поможет поддерживать необходимую влажность воздуха.\n- **Полив**: Полив должен быть регулярным, но не слишком частым. Земляк应该继续你的文章，用俄语完成它。以下是完整的俄文文章：\n\n## **Комнатное растение: Плантейн**\n\n**Плантейн**, или комнатный плантейн, является популярным выбором для начинающих цветоводов благодаря своей простоте в уходе и декоративности. Этот вид растения отличается крупными листьями зеленого цвета с серыми прожилками, которые придают ему уникальный внешний вид.\

Anchoring:  30%|███       | 3/10 [03:16<06:59, 59.92s/page]

...

content='## Нарцисс и его уход в домашних условиях\n\n**Нарцисс**, или кукурузник, является одним из самых популярных цветов для весеннего декора. Эти роскошные белые или желтые бутончики придают саду нежность и свежесть после зимы.\n\nДля успешного ухода за нарциссами в домашних условиях важно обеспечить им **достаточное освещение**. Лучше всего они растут на хорошо освещенных участках, но также могут переносить легкую тень. Влажность воздуха должна быть средней, избегайте чрезмерной влажности.\n\nПолив нарциссов должен быть регулярным, но не слишком частым. **Не переувлажняйте** корневую систему, так как это может привести к загниванию. В период активного роста можно использовать удобрения для цветущих растений.\n\nПосле цветения нарциссы лучше выкопать и хранить в прохладном месте до следующей весны. Таким образом, вы обеспечите им необходимый отдых перед новым циклом роста и цветения.' additional_kwargs={} response_metadata={'model': 'qwen2.5', 'created_at': '2025-03-01T21:38:17.17

Anchoring:  90%|█████████ | 9/10 [08:51<00:55, 55.32s/page]

content='## Гарденинские ленточки\n\n**Гарденинские ленточки** — это популярный способ ухода за домашними цветами, который помогает не только сохранить их красоту и здоровье, но и сделать ваш дом более уютным. Эти небольшие бусины из полимерной глины или других материалов можно прикреплять к веточкам растений для поддержки цветков и листьев.\n\n**Преимущества:** Гарденинские ленточки не только украшают растения, но и помогают улучшить их внешний вид. Они могут предотвратить скручивание лепестков и листьев, а также защитить цветы от ветра и других неблагоприятных условий.\n\n**Как использовать:** Просто прикрепите ленточки к веточкам растений с помощью обычного клея или специального фиксатора. Это не требует особых знаний, а результат будет заметен уже через несколько дней.\n\nГарденинские ленточки — это простой и эффективный способ ухода за вашими домашними цветами, который поможет им выглядеть более привлекательно и здоровыми.' additional_kwargs={} response_metadata={'model': 'qwen2.5

Anchoring: 100%|██████████| 10/10 [09:49<00:00, 58.95s/page]

content='## Tufted Hairgrass: Уход и Особенности Выведения\n\n**Тufted Hairgrass (Deschampsia cespitosa)** — это декоративное растение, которое часто используется в садовых композициях и аквариумах. Это многолетнее травянистое растение с мягкими волосистыми стеблями, которые придают саду нежный и спокойный вид.\n\n**Уход за тufted hairgrass:**  \n- **Освещение**: Предпочитает полутень или частичную тень.\n- **Влажность**: Требует умеренной влажности почвы, но не переносит переувлажнения.\n- **Почва**: Лучше всего растет на хорошо дренированной почве с нейтральным или слабокислым pH.\n\n**Особенности выведения:**  \nТufted Hairgrass легко размножается семенами и делением куста. Для лучшего внешнего вида рекомендуется ежегодное обрезание стеблей после цветения. Это растение хорошо сочетается с другими травянистыми культурами и создает идеальную среду для насекомых-опылителей.\n\n**Заключение:**  \nТufted Hairgrass — отличный выбор для тех, кто хочет добавить нежность и спокойствие в свой

In [58]:
df

,text
8f49ec5e-3bf4-4b4b-bcc7-2af3a8c217f7,## Матграсс: Уход за домашним растением\n\n**М...
00c01774-86a6-46f1-b4cb-b595a68576cd,## **Комнатное растение: Плантейн**\n\n**Плант...
040506ad-12e1-436d-a2dc-4d08e7eba2d5,## Cinquefoil: Уход за домашними цветами\n\n**...
06f2945a-47f6-4e4b-949b-31b32cf9ab1b,## Cow-parsnip: Уход за домашним растением\n\n...
082c6d57-1949-4336-ae11-8e73f88f3a60,## Mountain Geranium\n\n**Mountain geranium** ...
0a54b55b-6175-4750-8b9b-f132d8b1869c,## Quickbeam: Уход за домашней растительностью...
0f0394a9-138e-4154-a914-5e4d0f98277d,## Meadow Cress: Уход и Особенности Выведения\...
11b25fcd-3a66-4021-a1d1-677df8bf1477,## Нарцисс и его уход в домашних условиях\n\n*...
147bbcbb-b9df-45b2-9267-14087b957b26,## Гарденинские ленточки\n\n**Гарденинские лен...
14e0ec3e-4ac8-479b-b3b9-946df44e2c00,## Tufted Hairgrass: Уход и Особенности Выведе...


In [59]:
df.to_csv("articles.csv", index=True)